# Pythia 70 M 


## Find a Next Token Head


In [ ]:
import sys 
sys.path.append("../..")
sys.path.append("..")

from importlib import reload
from tqdm import tqdm

import joseph
from joseph.analysis import *
from joseph.visualisation import *
from joseph.utils import *
from joseph.data import *


reload(joseph.analysis)
reload(joseph.visualisation)
reload(joseph.utils)
reload(joseph.data)

from joseph.analysis import *
from joseph.visualisation import *
from joseph.utils import *
from joseph.data import *

# turn torch grad tracking off
torch.set_grad_enabled(False)

In [ ]:
import wandb
run = wandb.init()
artifact = run.use_artifact('jbloom/mats_sae_training_gpt2_small_hook_q_L4H11/sparse_autoencoder_gpt2-small_blocks.4.attn.hook_q_512:v4', type='model')
artifact_dir = artifact.download()

In [ ]:

model = HookedTransformer.from_pretrained(
    "gpt2-small",
    # "tiny-stories-2L-33M",
    # "attn-only-2l",
    # center_unembed=True,
    # center_writing_weights=True,
    # fold_ln=True,
    # refactor_factored_attn_matrices=True,
)
model.set_use_split_qkv_input(True)
model.set_use_attn_result(True)



# path = "./artifacts/sparse_autoencoder_gpt2-small_blocks.4.attn.hook_q_512:v4/500002816_sparse_autoencoder_gpt2-small_blocks.4.attn.hook_q_512.pt"
# path = "../checkpoints/4kydzgys/300003328_sparse_autoencoder_gpt2-small_blocks.4.attn.hook_q_2048.pt"
# path = "../checkpoints/lsl84wm6/750002176_sparse_autoencoder_gpt2-small_blocks.4.attn.hook_q_4096.pt"
path = "../checkpoints/lsl84wm6/final_sparse_autoencoder_gpt2-small_blocks.4.attn.hook_q_4096.pt"
sparse_autoencoder = SparseAutoencoder.load_from_pretrained(path)

print(sparse_autoencoder.cfg)


# sanity check
text = "Many important transition points in the history of science have been moments when science 'zoomed in.' At these points, we develop a visualization or tool that allows us to see the world in a new level of detail, and a new field of science develops to study the world through this lens."
model(text, return_type="loss")

In [ ]:

# Test on individual prompt (random repeating tokens)
random_tokens, random_token_groups = generate_random_token_prompt(model, n_random_tokens=16, n_repeat_tokens=2)
prompt = model.to_string(random_tokens)
print(prompt)

(logits, loss), cache = model.run_with_cache(prompt, return_type = "both", loss_per_token = True)
px.line(loss.squeeze().detach().cpu().numpy()).show()

from transformer_lens.head_detector import *

model.to("cpu")
pattern = get_previous_token_head_detection_pattern(model.to_tokens(prompt))
pattern = pattern.to("cpu")
px.imshow(detect_head(model, prompt, pattern), title="Previous token head detection").show()
# pattern = get_duplicate_token_head_detection_pattern(model.to_tokens(prompt).to("cpu"))
# pattern = pattern.to("cpu")
# px.imshow(detect_head(model, prompt, pattern), title="Duplicate token head detection").show()
# pattern = get_induction_head_detection_pattern(model.to_tokens(prompt).to("cpu"))
# pattern = pattern.to("cpu")
# px.imshow(detect_head(model, prompt, pattern), title="Duplicate token head detection").show()
model = model.to("mps")

In [ ]:
px.imshow(
    cache["blocks.4.attn.hook_pattern"][0].detach().cpu(),
    facet_col=0,
    facet_col_wrap=8,
).show()

In [ ]:
random_tokens, random_token_groups = generate_random_token_prompt(model, n_random_tokens=80, n_repeat_tokens=1)
prompt = model.to_string(random_tokens)
token_df, original_cache, cache_reconstructed_query, feature_acts = eval_prompt([prompt], model, sparse_autoencoder)
print(token_df.columns)
filter_cols = ["str_tokens", "unique_token", "context", "batch", "pos", "label", "loss", "loss_diff", "mse_loss", "num_active_features", "explained_variance", "kl_divergence"]
token_df[filter_cols].tail(10).style.background_gradient(
    subset=["loss_diff", "mse_loss","explained_variance", "num_active_features", "kl_divergence"],
    cmap="coolwarm",
)

In [ ]:
HEAD_IDX = sparse_autoencoder.cfg.hook_point_head_index
LAYER_IDX = sparse_autoencoder.cfg.hook_point_layer
patterns_original = original_cache[utils.get_act_name("attn_scores", LAYER_IDX)][0,HEAD_IDX].detach().cpu()
patterns_reconstructed = cache_reconstructed_query[utils.get_act_name("attn_scores", LAYER_IDX)][0,HEAD_IDX].detach().cpu()
both_patterns = torch.stack([patterns_original, patterns_reconstructed])
plot_attn(both_patterns.detach().cpu(), token_df, title="Original and Reconstructed Attention Distribution")
patterns_original = original_cache[utils.get_act_name("pattern", LAYER_IDX)][0,HEAD_IDX].detach().cpu()
patterns_reconstructed = cache_reconstructed_query[utils.get_act_name("pattern", LAYER_IDX)][0,HEAD_IDX].detach().cpu()
both_patterns = torch.stack([patterns_original, patterns_reconstructed])
plot_attn(both_patterns.detach().cpu(), token_df, title="Original and Reconstructed Attention Distribution")

In [ ]:
POS_INTEREST = feature_acts.shape[0] - 9 # index from 0.

# reload(joseph.visualisation)
# from joseph.visualisation import *

print(token_df["unique_token"][POS_INTEREST]) # bos gone.
plot_line_with_top_10_labels(feature_acts[POS_INTEREST], "", 10)

# vals, inds = torch.topk(feature_acts[POS_INTEREST],5)

# print(inds)
# plot_attn_score_by_feature(model, sparse_autoencoder, inds, original_cache, token_df, pos_interest=POS_INTEREST, vals = vals)
# plot_unembed_score_by_feature(model, sparse_autoencoder, inds, token_df, vals=vals)

# Analysis over Training Distribution

In [ ]:
data = get_webtext()

In [ ]:
str_token_list = []
loss_list = []
ablated_loss_list = []
# data = get_webtext()

NUM_PROMPTS = 10
# MAX_PROMPT_LEN = 100
# BATCH_SIZE = 10
dataframe_list = []
feature_acts_list = []
with torch.no_grad():
    for i in tqdm(range(NUM_PROMPTS)):
        
        # Get Token Data
        prompt = data[i]
        # new_str = data[BATCH_SIZE * i: BATCH_SIZE * (i + 1)]
        

        token_df, _, _, feature_acts = eval_prompt(prompt, model, sparse_autoencoder)
        feature_acts_list.append(feature_acts)
        dataframe_list.append(token_df)
        
all_token_df = pd.concat(dataframe_list)
all_token_df.reset_index(drop=True)
all_token_features = torch.cat(feature_acts_list)

print(all_token_df.shape)
print(all_token_df.columns)
all_token_df.head()

In [ ]:
pos_mask = (all_token_df.pos.values < 128)
all_token_df = all_token_df[all_token_df.pos < 128]
all_token_features = all_token_features[torch.tensor(pos_mask)]

In [ ]:
all_token_features.sum(dim=0).nonzero().shape

In [ ]:
all_token_df.shape

In [ ]:
sparsity = (all_token_features > 0).float().mean(dim=0).detach().cpu()
log_sparsity = torch.log10(sparsity)
print((sparsity > 0).float().mean())

In [ ]:
px.histogram(log_sparsity, nbins=100).show()
px.strip(log_sparsity, hover_data=[[i for i in range(sparse_autoencoder.cfg.d_sae)]]).show()

In [ ]:
all_token_df.pos = all_token_df.pos.astype(str)
# get some random features
random_features = torch.randint(0, 1024, (3,))
for feature in torch.tensor([235,702,1540]):
    all_token_df[f"feature_{feature}"] = all_token_features[:,feature].detach().cpu()
    feature_density = (all_token_df[f"feature_{feature}"] > 0).mean()
    if feature_density > 0:
        print(f"Feature {feature} has density {feature_density}")
        px.scatter(
            all_token_df,
            x = "pos",
            y = f"feature_{feature}",
            color="pos",
            hover_data=["unique_token", "context", "label"]
            ).show()


# Exploratory

In [ ]:
position = 72
mask = [i  % 128 == position  for i in range(all_token_df.shape[0])]
scores = (all_token_features[mask, :]>0).float().mean(dim=0)
vals, inds = torch.topk(scores, 10)
all_token_df[mask].head()


In [ ]:
plot_line_with_top_10_labels(scores, "", 10)
# plot_line_with_top_10_labels(all_token_features[position,], "", 3)
# plot_line_with_top_10_labels(all_token_features[position+128,], "", 3)
# plot_line_with_top_10_labels(all_token_features[position+128*2,], "", 3)

In [ ]:
filter_columns = ["pos", "unique_token", "context", "label", "loss", "loss_diff", "mse_loss", "num_active_features", "explained_variance", "kl_divergence"]
for feature in torch.tensor(inds):
    all_token_df[f"feature_{feature}"] = all_token_features[:,feature].detach().cpu()
    mean_activation = (all_token_df[f"feature_{feature}"] > 0).mean()
    if mean_activation > 0.001:
        print(feature.item(), (all_token_df[f"feature_{feature}"] > 0).mean()) # super super dense feature.
        display(all_token_df.sort_values(f"feature_{feature}", ascending=False)[filter_columns + [f"feature_{feature}"]].head(5))
    else:
        print(f"feature {feature.item()} is dead")

# Causal

In [ ]:
def test_get_attn_sae_feature_removal(example_prompt, 
                                      example_prompt_answer, 
                                      model, 
                                      sparse_autoencoder,
                                      features_to_remove=[0],
                                      token_unembed_to_remove=[]):
    
    layer_index = sparse_autoencoder.cfg.hook_point_layer
    head_index = sparse_autoencoder.cfg.hook_point_head_index
    
    example_prompt_tokens = model.to_tokens(example_prompt)
    attn_df, original_cache, cache_reconstructed_query, feature_acts = eval_prompt([example_prompt], model, sparse_autoencoder)

    original_acts = original_cache[sparse_autoencoder.cfg.hook_point]
    print(original_acts.shape)
    sae_out, feature_acts, loss, mse_loss, l1_loss = sparse_autoencoder(
       original_acts[0,:,head_index]
    )
    

    def remove_feature_hook(hook_in, hook, head = head_index, features_to_remove = features_to_remove):
        for feature_to_remove in features_to_remove:
            feature_dir = feature_acts[-1,feature_to_remove]*sparse_autoencoder.W_dec[feature_to_remove]
            hook_in[:, :, head] -= feature_dir
        return hook_in
    
    with model.hooks(fwd_hooks=[(sparse_autoencoder.cfg.hook_point, remove_feature_hook)]):
        _, cache_removed_feature = model.run_with_cache(example_prompt_tokens, return_type="loss", loss_per_token=True)
        
        
    def replace_with_feature_hook(hook_in, hook, head = head_index, features_to_add = features_to_remove):
        new_act = torch.zeros_like(hook_in[:, :, head])
        for features_to_add in features_to_remove:
            feature_dir = feature_acts[-1,features_to_add]*sparse_autoencoder.W_dec[features_to_add]
            new_act += feature_dir
        hook_in[:, :, head] = new_act
        return hook_in
    
    with model.hooks(fwd_hooks=[(sparse_autoencoder.cfg.hook_point, replace_with_feature_hook)]):
        _, cache_only_features = model.run_with_cache(example_prompt_tokens, return_type="loss", loss_per_token=True)
        

    def remove_token_unembed_hook(hook_in, hook, head = head_index, token_unembed_to_remove = token_unembed_to_remove):
        for token_id in token_unembed_to_remove:
            unembed_dir = model.W_U[:, token_id] @ model.W_Q[LAYER_IDX,head_index]
            unembed_dir_proj =  hook_in[:, :, head] @ unembed_dir.T
            hook_in[:, :, head] -= unembed_dir_proj[:,:,None]*unembed_dir
        return hook_in
    
    with model.hooks(fwd_hooks=[(sparse_autoencoder.cfg.hook_point, remove_token_unembed_hook)]):
        _, cache_removed_token_unembed = model.run_with_cache(example_prompt_tokens, return_type="loss", loss_per_token=True)

    patterns = original_cache[f"blocks.{layer_index}.attn.hook_pattern"][0,head_index].detach().cpu()
    attn_df = make_token_df(model, example_prompt_tokens)
    # remove last row of attn df
    attn_df["original_attn"] = patterns[-1,]
    patterns = cache_reconstructed_query[f"blocks.{layer_index}.attn.hook_pattern"][0,head_index].detach().cpu()
    attn_df["reconstructed_attn"] = patterns[-1,]
    patterns = cache_removed_feature[f"blocks.{layer_index}.attn.hook_pattern"][0,head_index].detach().cpu()
    attn_df["ablated_feature_attn"] = patterns[-1,]
    patterns = cache_only_features[f"blocks.{layer_index}.attn.hook_pattern"][0,head_index].detach().cpu()
    attn_df["cache_only_features"] = patterns[-1,]
    patterns = cache_removed_token_unembed[f"blocks.{layer_index}.attn.hook_pattern"][0,head_index].detach().cpu()
    attn_df["cache_removed_token_unembed"] = patterns[-1,]
    
    if len(token_unembed_to_remove) > 0:
        fig = px.line(attn_df, 
                    y=["original_attn","reconstructed_attn", "ablated_feature_attn", "cache_only_features", "cache_removed_token_unembed"],
                    hover_name="str_tokens", 
                    hover_data=["pos", "batch", "label"], 
                    title="Original vs Reconstructed attention")
    else:
        fig = px.line(attn_df, 
                    y=["original_attn","reconstructed_attn", "ablated_feature_attn", "cache_only_features"],
                    hover_name="str_tokens", 
                    hover_data=["pos", "batch", "label"], 
                    title="Original vs Reconstructed attention")
    
    # increase figure height
    fig.update_layout(height=800)
    fig.show()


random_tokens, random_token_groups = generate_random_token_prompt(model, n_random_tokens=4, n_repeat_tokens=2)
example_prompt = model.to_string(random_tokens[:-1])
example_prompt_answer = model.to_string(random_tokens[:-1])
attn_df, original_cache, cache_reconstructed_query, feature_acts = eval_prompt([example_prompt], model, sparse_autoencoder)
attn_df.tail(2).T


In [ ]:
patterns_original = original_cache[utils.get_act_name("pattern", LAYER_IDX)][0,HEAD_IDX].detach().cpu()
patterns_reconstructed = cache_reconstructed_query[utils.get_act_name("pattern", LAYER_IDX)][0,HEAD_IDX].detach().cpu()
both_patterns = torch.stack([patterns_original, patterns_reconstructed])
plot_attn(both_patterns.detach().cpu(), attn_df, title="Original and Reconstructed Attention Distribution")

POS_INTEREST = feature_acts.shape[0] - 1 # index from 0.
print(attn_df["unique_token"][POS_INTEREST]) # bos gone.
plot_line_with_top_10_labels(feature_acts[POS_INTEREST], "", 10)


In [ ]:
#  673, 3252, 3218, 1518, 1344, 1185, 311
test_get_attn_sae_feature_removal(
    example_prompt, 
    example_prompt_answer, model, 
    sparse_autoencoder, features_to_remove=[673])


# test_get_attn_sae_feature_removal(
#     example_prompt, 
#     example_prompt_answer, model, 
#     sparse_autoencoder, features_to_remove=[1344])


# Directly Search for Positional Features

In [ ]:
# how can we automatically match features to positions?
# for position in position:
#     for feature in features: -> get recall and precision

positions = list(range(128)) # for the first 10 positions
features = list(range(sparse_autoencoder.cfg.d_sae)) # check all features

all_scores = []

for position in tqdm(positions):
    for feature in features:
        mask = [i  % 128 == position  for i in range(all_token_df.shape[0])]
        score = all_token_features[mask, feature].mean().item()
        all_scores.append([position, feature, score])
        
        
scores_df = pd.DataFrame(all_scores, columns=["pos", "feature", "score"])


In [ ]:
# get the top positiion for each feature
# get the top score per feature (criteria: activates at that pos)
top_feature_pos_scores_df = scores_df.groupby(["feature","pos"]).score.mean().reset_index().sort_values("score", ascending=False).groupby("feature").head(1).sort_values("pos")
# then for each pos, get the features who's maximal top score is at that position (criteria: activates more than anything else)
top_feature_pos_scores_df = top_feature_pos_scores_df.sort_values("score", ascending=False).groupby("pos").head(3).sort_values("pos")
top_feature_pos_scores_df.head()

In [ ]:
tmp = scores_df.pivot(index="pos", columns="feature", values="score")
print(tmp.shape)
px.imshow(tmp, color_continuous_scale="RdBu_r", title="Feature Scores by Position").show()
tmp = pd.DataFrame(
    tmp.loc[:,top_feature_pos_scores_df.feature.values].values,
    index = list(tmp.index.to_flat_index()),
    columns = [str(i) for i in list(top_feature_pos_scores_df.feature)],
)
px.imshow(tmp.T, color_continuous_scale="RdBu_r", title="Feature Scores by Position").show()

In [ ]:
px.line(tmp[["2674"]]).show()

In [ ]:
px.line(tmp)
# px.line(tmp / tmp.values.max(axis=0, keepdims=True))

wandb: Network error (ConnectionError), entering retry loop.


In [ ]:
# px.line(tmp[["430"]]).show()
px.line(tmp[["1012","241","1703", "280"]]).show()
# px.line(tmp[["545"]]).show()

# Clustering

In [ ]:
from scipy.cluster import hierarchy

def plot_cosine_similarity_heatmap(df, restricted_labels, reorder = False, title="Pairwise Cosine Similarity Heatmap"):
    data_array = df.to_numpy()
    linkage = hierarchy.linkage(data_array)
    dendrogram = hierarchy.dendrogram(
        linkage, no_plot=True, color_threshold=-np.inf
    )
    if reorder:
        reordered_ind = dendrogram["leaves"]
        # reorder df by ind
        df = df.iloc[reordered_ind, reordered_ind]
        # data_array = df.to_numpy()

    # plot the cosine similarity matrix
    fig  = px.imshow(
            df.values,
            color_continuous_scale="RdBu",
            color_continuous_midpoint=0.0,
            labels={"color": "Cosine Similarity"},
        )
    fig.update_xaxes(
        tickmode="array",
        tickvals=list(range(len(restricted_labels))),
        ticktext=restricted_labels,
        showgrid=False,
    )
    fig.update_yaxes(
        tickmode="array",
        tickvals=list(range(len(restricted_labels))),
        ticktext=restricted_labels,
        showgrid=False,
    )

    # don't show axes if there are more than 20 rows 
    if df.shape[0] > 20:
        fig.update_xaxes(
            visible=False,
        )
        fig.update_yaxes(
            visible=False,
        )
    return fig

feature_dirs = pd.DataFrame(sparse_autoencoder.W_dec.detach().cpu()).reset_index(drop=True).T.corr()
plot_cosine_similarity_heatmap(feature_dirs, restricted_labels=[i for i in range(sparse_autoencoder.cfg.d_sae)], reorder =True).show()

## what about non-positional features?

In [ ]:
import hdbscan
clusterer = hdbscan.HDBSCAN(min_cluster_size=11)
tmp = top_feature_pos_scores_df.sort_values("score", ascending=True).head(256)
tmp["mean_activation"] = tmp.feature.apply(lambda x: all_token_features[:,x].mean().item())
clusterer.fit(tmp[["score", "mean_activation"]])
tmp["group"] = [f"group_{i}" for i in clusterer.labels_]
px.scatter(tmp, x="score", y="mean_activation", color="group",
           hover_data=["feature"])

In [ ]:
filter_columns = ["pos", "unique_token", "context", "label", "loss", "loss_diff", "mse_loss", "num_active_features", "explained_variance", "kl_divergence"]
for feature in torch.tensor([9, 545, 42]):
    all_token_df[f"feature_{feature}"] = all_token_features[:,feature].detach().cpu()
    print(feature.item(), (all_token_df[f"feature_{feature}"] > 0).mean()) # super super dense feature.
    display(all_token_df.sort_values(f"feature_{feature}", ascending=False)[filter_columns + [f"feature_{feature}"]].head(10))

In [ ]:
import umap 
import hdbscan

reducer = umap.UMAP(
    n_neighbors=8,
    min_dist=0.15,
    n_components=2,
    metric="cosine",
    random_state=42,
)

clusterer = hdbscan.HDBSCAN(min_cluster_size=4)


ummap_result = reducer.fit_transform(feature_acts.T.detach().cpu())
# ummap_result = reducer.fit_transform(sparse_autoencoder.W_dec.detach().cpu())
clusterer.fit(ummap_result)
print(clusterer.labels_)


fig = px.scatter(
        ummap_result, 
        x= 0,
        y=1,
        color=[str(i) for i in clusterer.labels_],
        hover_data = [[f"feature_{i}" for i in range(sparse_autoencoder.cfg.d_sae)]],
        opacity=0.5,
        template="plotly",
    )

# make points larger
fig.update_traces(marker=dict(size=12))

# make it wide and tall
fig.update_layout(height=800, width=1200)
    
fig.show()

In [ ]:
# set pandas col width limit
pd.set_option('display.max_colwidth', 200)

groups = pd.DataFrame(ummap_result, columns=["x","y"])
groups["group"] = [f"{i}" for i in clusterer.labels_]
groups["feature"] = [f"{i}" for i in range(sparse_autoencoder.cfg.d_sae)]
groups = groups[["feature", "group", "x", "y"]]


feature_group = "41"
features = groups[groups.group == feature_group].feature.astype(int).values
for feature in torch.tensor(features):
    all_token_df[f"feature_{feature}"] = all_token_features[:,feature].detach().cpu()
    mean_activation = (all_token_df[f"feature_{feature}"] > 0).mean()
    if mean_activation > 0.001:
        print(feature.item(), (all_token_df[f"feature_{feature}"] > 0).mean()) # super super dense feature.
        display(all_token_df.sort_values(f"feature_{feature}", ascending=False)[filter_columns + [f"feature_{feature}"]].head(5))
    else:
        print(f"feature {feature.item()} is dead")

In [ ]:
import umap 
import hdbscan

reducer = umap.UMAP(
    n_neighbors=2,
    min_dist=0.12,
    n_components=2,
    metric="euclidean",
    random_state=42,
)

clusterer = hdbscan.HDBSCAN(min_cluster_size=5)


ummap_result = reducer.fit_transform(sparse_autoencoder.W_dec.detach().cpu())
clusterer.fit(ummap_result)
# print(clusterer.labels_)


fig = px.scatter(
        ummap_result, 
        x= 0,
        y=1,
        color=[str(i) for i in clusterer.labels_],
        hover_data = [[f"feature_{i}" for i in range(sparse_autoencoder.cfg.d_sae)]],
        opacity=0.5,
        template="plotly",
    )

# make points larger
fig.update_traces(marker=dict(size=12))

# make it wide and tall
fig.update_layout(height=800, width=1200)
    
fig.show()

In [ ]:
# set pandas col width limit
pd.set_option('display.max_colwidth', 200)

groups = pd.DataFrame(ummap_result, columns=["x","y"])
groups["group"] = [f"{i}" for i in clusterer.labels_]
groups["feature"] = [f"{i}" for i in range(sparse_autoencoder.cfg.d_sae)]
groups = groups[["feature", "group", "x", "y"]]


feature_group = "229"
features = groups[groups.group == feature_group].feature.astype(int).values
for feature in torch.tensor(features):
    all_token_df[f"feature_{feature}"] = all_token_features[:,feature].detach().cpu()
    mean_activation = (all_token_df[f"feature_{feature}"] > 0).mean()
    if mean_activation > 0.001:
        print(feature.item(), (all_token_df[f"feature_{feature}"] > 0).mean()) # super super dense feature.
        display(all_token_df.sort_values(f"feature_{feature}", ascending=False)[filter_columns + [f"feature_{feature}"]].head(5))
    else:
        print(f"feature {feature.item()} is dead")